---
syncID: 
title: "Stacking and Joining NEON Data with Python"
description: ""
dateCreated: 2020-05-07
authors: Maxwell J. Burner
contributors: Donal O'Leary
estimatedTime: 
packagesLibraries: requests, json, pandas
topics: api, data management, reshaping data
languagesTool: python
dataProduct: DP1.10003.001
code1: 
tutorialSeries: python-neon-api-series
urlTitle: python-neon-api-06-stacking
---

In this tutorial we will learn how to stack and join NEON data tables using the Pandas library.

<div id="ds-objectives" markdown="1">

### Objectives
After completing this tutorial, you will be able to:

* Combine NEON data tables from different sites and months using Pandas *concat* function
* Describe the difference between inner joins, left outer joins, right outer joins, and full outer joins
* Combine NEON data tables of different types using Pandas *merge* method


### Install Python Packages

* **requests**
* **json** 
* **pandas**


</div>

In this tutorial we will learn how to combine different two or more tables loaded as Pandas dataframes into one. The NEON API returns data in separate tables for each month and site. Furthermore, a data product package for one site and month usually includes mutiple tables, related to each other but storing different variables. 

As a result, the data we want for a particular study may be spread across multiple tables. In order to effectively manipulate our data in Python, we will usually want to combine these tables into one. When combining data tables of the same type, we often call this process *stacking*, since the rows of each table are "stacked" on the rows of the others. When combining tables of different types, we call this *joining*, and the observations in each table become a related set of observations. 

Fortunately, the Pandas package includes functions and methods for both *stacking* and *joining*  data frames. Here we will discuss stacking data frames using a fairly simple concatenation method, and the more complex subject of joining dataframes. For our examples, we will again use breeding landbird count data from NEON's lower Teakettle site.

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
SERVER = 'http://data.neonscience.org/api/v0/'
SITECODE = 'TEAK'
PRODUCTCODE = 'DP1.10003.001'

## Stacking tables

As we have seen, NEON stores data of the same type in seperate tables by site and month. But what if we want data spanning multiple months, or from multiple sampling sites? If the data tables are of the same 'type' - such as basic package bird count data - they will have the same columns and headers, so we can simply combine rows from multiple tables into one.

From our first tutorial, we know that bird count data taken at Lower Teakettle is available for several months, including both June and July of 2019. For our example here, we will merge the basic bird count data tables for those two months into one.

In [ ]:
#Define a function that takes a 'data/' endpoint json and downloads a csv file from it based on provided strings
#Input: A json object with results of a NEON 'data/' endpoint API call, two strings indicating file name
#Output: A pandas dataframe
def get_data(data_json, string1, string2 = ''):
    for file in data_json['data']['files']:
        if(string1 in file['name']):
            if(string2 in file['name']):
                return pd.read_csv(file['url'])
    print('No files matching name')
    return(0)
 

In [ ]:
#Request information on data
june_req = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2019-06')
july_req = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2019-07')

june_json = june_req.json()
july_json = july_req.json()

In [ ]:
#Read in basic bird count data for June and July 2019 of lower Teakettle, using the function we defined.
df_june_count = get_data(june_json, 'countdata', 'basic')
df_july_count = get_data(july_json, 'countdata', 'basic')

In [ ]:
#View first three rows of June data
df_june_count.head(3)

In [ ]:
#View first three rows of July count data
df_july_count.head(3)

In [ ]:
#View shape of each count data table
print('June rows and columns: ',df_june_count.shape)
print('July rows and columns: ',df_july_count.shape)

The two dataframes have matching number of columns, column names, and column data types. We can directly combine them into one child table containing the rows of both parent tables using the Pandas *concat* function. This takes a list, tuple, or other sequence of data frames, and tries to combine them into one.

The *concat* function can combine rows, concatenating along the row indices, or it can combine columns, concatenating along the header. Which approach is used is determined by the *axis* parameter; 0 for rows, 1 for columns. The default is 0 for rows, so in this case we don't have to specify.

In [ ]:
#Combine data frames along index
df_concatenated = pd.concat((df_june_count, df_july_count))
print(df_concatenated.shape)

The new dataframe has 1770 rows, the sum of the 1484 rows from the June data and the 286 rows from the July data.

## Joining Tables

At other times, the data we want may be spread across two or more related tables, each with different attributes. In this case we will want to **join** the tables together. Joining tables is an important concept in data science and analysis. It requires that we provide a join predicate, so Python knows whether a row in one table should be associated with row(s) in the second table. This is often done by specifying a unique ID column (a key) that tie the observations or data, and this key must be found in both tables. Several different kinds of joins exist, depending on the relationship between the tables and how we want to handle unmatched data. 

The breeding land bird count data comes with several tables, including the 'Count Data' table (which contains all of the different individual bird observations and their taxonomic information) and the 'Point Data' table (which contains information about the point where the birds were observed (spatial information, vegetation type, etc.) as well as the observation conditions for the day (cloud cover, wind speed, relative humidity, etc.). In order to match up every bird observation with the location and conditions in which it was observed, we will need to join these two tables together.

For this example, we will download the per point data for June as a dataframe, and join it with the count data for June. 

In [ ]:
#Import basic package per point data into Python
df_june_point = get_data(june_json, 'perpoint', 'basic')

#View shape of new dataframe
print(df_june_point.shape)
df_june_point.head(3)

In [ ]:
#View columns of count data table
print('Count data')
print(df_june_count.dtypes)

print('\nPoint Data')
print(df_june_point.dtypes)

Notice that the countdata table and the perpoint table share a number of common columns/variables/attributes. We say that a *relationship* exists between these tables; data in a certain column of one table corresponds to data in a certain column of the other table. To join two tables, we select a pair of related columns from the tables to serve as the *keys* in the join.

One of the columns present in both tables is 'eventID'. Do the values of this column in one table appear to correspond to those in the other table?

In [ ]:
#Print first ten rows of eventID column for each table
for i in range(10):
    print('Point: ',df_june_count['eventID'][i],'\tCount: ' ,df_june_point['eventID'][i])

It appears that it should be possible to match values between the two eventID columns; in fact, the two columns have identical first entries.

If we are using just one key pair (e.g. df_june_count['eventID'] paired with df_june_point['eventID']), we ideally want there to be no repeated values in the key column of the table with fewer rows. This way, every row in the longer table matches to no more than one row in the shorter table.

Here the shorter table is the perpoint data; let's see if the eventID attribute has only unique values.

In [ ]:
#Compare shape of whole eventID column to shape of eventID column after removing repeated values.
print(df_june_point['eventID'].shape)
print(pd.unique(df_june_point['eventID']).shape)


It looks like there are no repeat values in the eventID column of the per point table. Combined with the fact that the eventID values in the count data table correspond to eventID values in the per point table, we can use this as a key. Our join will use every row of the longer count data table only once, and will duplicate rows of the shorter per point data table as necessary so that the number of rows match. Our final table will have the same number of rows as the per point table.

The next question is what kind of join we want; this determines how the join algorithm deals with rows in one table that lack a matching row in the other. If we perform an **Inner Join**, any rows from either table that lack a match in the other will be left out of the final table; if all but three rows of the longer table have matching rows in the shorter table, then final table will have three rows fewer than the longer parent table.

<figure>
    <img src = https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png>
    <figcaption>Source: R for Data Science. Image licensed under Creative Commons. <a href="https://r4ds.had.co.nz/">Credit: Wickham and Grolemund, R for Data Science</a> </figcaption>
</figure>

If we instead perform an **Outer Join**, then rows that lack a match will be allowed to stay in the final table; the added columns will be "padded" with NA values in the output table. If we only keep unmatched row from the left table it is a *Left Outer Join* or **Left Join**; if we only keep unmatched rows from the right table it is a *Right Outer Join* or **Right Join**; and if we keep unmatched rows from both tables it is a *Full Outer Join* or **Full Join**.

<figure>
    <img src = https://d33wubrfki0l68.cloudfront.net/9c12ca9e12ed26a7c5d2aa08e36d2ac4fb593f1e/79980/diagrams/join-outer.png>
    <figcaption>Source: R for Data Science. Image licensed under Creative Commons. <a href="https://r4ds.had.co.nz/">Credit: Wickham and Grolemund, R for Data Science</a></figcaption>
</figure>

In this example we will use an inner join, as we don't want any rows with missing values in the key fields. The final table will have the same number of rows as the count data table, the longer table, minus any rows that didn't have a match in the point data table.

The next question is which columns from each table we want to include in the final table. The per point data table has quite a few columns that are related to or identical to columns in the count data table; bringing these along would add redundant data.

In [ ]:
#View columns of June count data table
print(df_june_count.dtypes)

In [ ]:
#View columns of June per point data table
df_june_point.dtypes

The two dataframes share the columns 'namedLocation', 'domainID', 'siteID', 'plotID', 'plotType', 'pointID', 'startDate', and 'eventID'. Both tables also have a 'uid' column, but these are just values used to index each row in a table; the 'uid' values in the count data table are not related to the 'uid' values in the point data table.

Our merge will bring over values from all of the point data table column that aren't already present in the point data table. We also won't bring over 'uid' values from the point data table; these would not be of any use in the merged table.



In [ ]:
#Create a list of columns to keep; columns only present in the point data, plus eventID
good_columns = []
for column in df_june_point:
    if(column not in df_june_count.columns):
        good_columns.append(column)
        
good_columns.append('eventID')
print(good_columns)

The Pandas package offers a couple of different methods for joining data frames. In this case we will use the **.merge** method of dataframes. This method is called on one 'left' dataframe, and is passed another 'right' dataframe, along with further parameters indicating how the join is to occur: *left_on* and *right_on* indicate the column from the left and right dataframes to be used as keys respectively, and the *how* parameter is given a string 'inner' (default), 'left', 'right', or 'full' that determines the type of join performed.

You can read the Pandas documentation for *pandas.Dataframe.merge* [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) (pandas.pydata.org). Pandas also offers a *.join* method for the same purpose; that method has fewer parameters, but assumes that the key column in the right table is that table's index.

Now we make the join. First we will use the *.filter* method and the list we prepared to get a version of the point data table that only contains the desired columns. Then we call *.merge* on the count data, and pass the prepared data table.

In [ ]:
#Prepare point dataframe for joining by narrowing to only the desired columns
df_right = df_june_point.filter(items = good_columns, axis = 1)

In [ ]:
#Join data frames
df_joined = df_june_count.merge(df_right, how = 'inner', left_on = 'eventID', right_on = 'eventID')
print(df_joined.columns)

Finally, let's check the shape of the joined data table against the original count data table.

In [ ]:
print(df_june_count.shape)
print(df_joined.shape)

The number of rows is the same in each, so it appears that every entry in the count data was successfully mapped to an entry in the point data.

Finally, let's take a look at the new joined dataframe to check our work.

In [ ]:
pd.set_option('max_columns', 40) #show all columns
df_joined.head(3)

We see that now every individual bird observation is listed with the associated location and observation conditions; just as we wanted.

If you want to learn more about this subject, the Pandas package's official site offers an in-depth [discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) (pandas.pydata.org) of using *concat*, *join*, *merge*, and other functions to combine data frames.